In [ ]:
!python3 -m pip install tensorflow==2.1.0
# This cell has the latest set up for AI Platform

In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import tensorflow as tf
from tensorflow.keras import Model
#from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
import os
import glob
import cv2
from io import BytesIO
from PIL import Image
from numpy import expand_dims
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.callbacks import *
import warnings
import logging
from IPython.display import clear_output
from collections import Counter
import pickle
import sys
sys.path.insert(1, '/home/jupyter/DeepFake-2019-20/visualisations')
import VisualisationTools as plotting

import TransformCode as tc

warnings.filterwarnings("ignore")
logger = logging.getLogger()
logger.setLevel(100)
plot_losses = plotting.PlotLearning()
os.chdir('/home/jupyter/DeepFake-2019-20')


print("Tensorflow version:", tf.__version__)

Tensorflow version: 2.1.0


In [2]:
import os
import glob
import keras
import VideoFrameGenerator
# use sub directories names as classes
classes = [i.split(os.path.sep)[-1] for i in glob.glob('../restructured-all-faces/home/jupyter/restructured_data/train/*')]
classes.sort() # actually already within source code
print(classes)
# some global params
SIZE = (224, 224)
CHANNELS = 3
NBFRAME = 20
BS = 8
# pattern to get videos and classes

glob_pattern_train = '../restructured-all-faces/home/jupyter/restructured_data/train/{classname}/*'
glob_pattern_val = '../restructured-all-faces/home/jupyter/restructured_data/validation/{classname}/*'

#glob_pattern='videos/{classname}/*.avi'
# for data augmentation
data_aug = tf.keras.preprocessing.image.ImageDataGenerator(
    zoom_range=.1,
    horizontal_flip=True,
    rotation_range=8,
    width_shift_range=.2,
    height_shift_range=.2)

# Create video frame generator

train = VideoFrameGenerator.VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern_train,
    nb_frames=NBFRAME,
#    split=0, 
    shuffle=True,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    transformation=data_aug,
    use_frame_cache=False)

val = VideoFrameGenerator.VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern_val,
    nb_frames=NBFRAME,
#    split=0, 
    shuffle=True,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    transformation=data_aug,
    use_frame_cache=False)

Using TensorFlow backend.


['authentic', 'fake']
Total data: 2 classes for 3600 files for train
Total data: 2 classes for 700 files for train


### Watch out for USE_HEADER!

In [3]:
from tensorflow.keras.preprocessing.image import \
    img_to_array
# Watch out for USE_HEADER!
#img_to_array('../restructured-all-faces/home/jupyter/restructured_data/train/authentic/Original_001/0000.png')

In [4]:
#VideoFrameGenerator.utils.show_sample(train)
print(train.indexes[0:30])
# what are indexes?
print(train.classes_count)
print(train.on_epoch_end)

[3221   12  856 2893 2660 1487  713 1716 3171 1070  643   80 3391  212
 1234 1794  855 3263 2810  930 2569 1345 1552  711 2865 2052  421 2728
 1820 1317]
2
<bound method VideoFrameGenerator.on_epoch_end of <VideoFrameGenerator.VideoFrameGenerator object at 0x7fa526ac3dd0>>


In [5]:
word = '../restructured-all-faces/home/jupyter/restructured_data/train/authentic/Original_001'
word + '/' + (sorted(os.listdir(word))[0])

'../restructured-all-faces/home/jupyter/restructured_data/train/authentic/Original_001/0000.png'

In [6]:
print(train.files[0:4])
#print(train._framecounters)
print(train.nbframe)

['../restructured-all-faces/home/jupyter/restructured_data/train/authentic/Original_397', '../restructured-all-faces/home/jupyter/restructured_data/train/authentic/Original_997', '../restructured-all-faces/home/jupyter/restructured_data/train/authentic/Original_692', '../restructured-all-faces/home/jupyter/restructured_data/train/authentic/Original_627']
20


# Code below hopefully now relevant

In [9]:
def build_model(dropout, lr_rate, architecture = 'lstm'):
    frames = 20
    channels = 3
    rows = 224
    columns = 224
    
    embed_dim = 32  # Embedding size for each token
    num_heads = 2  # Number of attention heads
    ff_dim = 32  # Hidden layer size in feed forward network inside transformer
    
    video = tf.keras.layers.Input(shape=(frames,
                         rows,
                         columns,
                         channels,))
    
    
    if architecture == 'lstm':
        
        from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
        conv_base = MobileNetV2(weights='imagenet', include_top=False,
                            input_shape=(224,224,3))
    
        cnn_out = GlobalAveragePooling2D()(conv_base.output)
        cnn = tf.keras.Model(inputs=conv_base.input, outputs=cnn_out)
        cnn.trainable = False
    
        encoded_frames = tf.keras.layers.TimeDistributed(cnn)(video)
        
        encoded_sequence = tf.keras.layers.LSTM(256)(encoded_frames)
        hidden_layer = Dense(128, activation="relu")(encoded_sequence)
        outputs = Dense(2, activation="softmax")(hidden_layer)
        model = Model([video], outputs)
    
        
    if architecture == 'transformer':
        
        embedding_layer = tc.TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
        x = embedding_layer(cnn.output)
        transformer_block = tc.TransformerBlock(embed_dim, num_heads, ff_dim)
        x = transformer_block(x)
        x = tf.keras.layers.GlobalAveragePooling1D()(x)
        x = tf.keras.layers.Dropout(0.1)(x)
        x = tf.keras.layers.Dense(20, activation="relu")(x)
        x = tf.keras.layers.Dropout(0.1)(x)
        outputs = tf.keras.layers.Dense(2, activation="softmax")(x)
        model = Model([video], outputs)
                
        
    
    optimizer = tf.keras.optimizers.Nadam(lr=lr_rate,
                      beta_1=0.9,
                      beta_2=0.999,
                      epsilon=1e-08,
                      schedule_decay=0.004)
    model.compile(loss="binary_crossentropy",
                  optimizer=optimizer,
                  metrics=["binary_accuracy"]) 
    return model
model = build_model(0.1,0.002)

In [10]:
EPOCHS = 1
model.fit_generator(
    train,
    validation_data=val,
    verbose=1,
    epochs=EPOCHS#,
  #  callbacks=callbacks
)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 450 steps, validate for 87 steps
 10/450 [..............................] - ETA: 45:35 - loss: 0.5179 - binary_accuracy: 0.8056

KeyboardInterrupt: 